# Conversion of Lausanne grid data from Lu

In [ ]:
# Import packages
import os
import logging
from pathlib import Path
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import unary_union
import matplotlib.pyplot as plt

logging.basicConfig(level=logging.INFO,
                    format='[%(levelname)s] %(funcName)s: %(message)s',
                    )

In [ ]:
shared_folder = '/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025'
grid_path = os.path.join(shared_folder, 'data/sebeillon-node-loc.csv')
buildings_path = '/Users/catarina/switchdrive/IPESE-DESL_data_exchange/Weather/CRYOS_data/buildings_cryos.gpkg'

def generate_gdf(point_path, long_col, lat_col, input_crs):
    """
    Function to generate a geodataframe from a csv file with point data.
    Parameters:
    - point_path: str, path to the csv file
    - long_col: str, name of the column with the longitude data
    - lat_col: str, name of the column with the latitude data
    - crs: int, EPSG code of the coordinate reference system
    Returns:
    - gdf: geopandas geodataframe
    """
    # check if extension is csv
    if Path(point_path).suffix == '.csv':
        # read csv
        df = pd.read_csv(point_path)
        # create geometry column
        df['geometry'] = gpd.points_from_xy(df[long_col], df[lat_col])
        # create geodataframe
        gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=input_crs)
        return gdf
    else:
        print('File extension not supported. Please provide a csv file.')

def generate_grid_mask(grid_gdf, offset, crs_output):
    """
    Function to generate a polygon mask from a line that encircles the points from a GeoDataFrame plus an offset.
    
    Parameters:
    - grid_gdf: geopandas GeoDataFrame, grid points
    - offset: int, buffer offset in meters
    - crs_output: int, EPSG code of the output coordinate reference system
    
    Returns:
    - mask: geopandas GeoDataFrame, polygon mask of the grid
    """
    # Create a convex hull around the points
    hull = unary_union(grid_gdf.geometry).convex_hull

    # Apply buffer (offset in meters)
    mask_polygon = hull.buffer(offset)

    # Convert back to the original CRS if needed
    mask_gdf = gpd.GeoDataFrame(geometry=[mask_polygon], crs=grid_gdf.crs)
    
    # Reproject to the specified output CRS if necessary
    if crs_output is not None and mask_gdf.crs.to_epsg() != crs_output:
        mask_gdf = mask_gdf.to_crs(epsg=crs_output)

    return mask_gdf

def clean_qbuildings(buildings_path,crs):
    """
    Checks if there are empty columns and nan values that might give errors when running reho, and removes them.
    If necessary, converts the coordinate's reference system to the one from the temperature file
    Returns:
        buildings_gdf: Cleaned buildings GeoDataFrame
    """
    
    # read buildings
    buildings_gdf = gpd.read_file(buildings_path)

    # Initial row count
    build_ini = buildings_gdf.shape[0]
    logging.info(f'reading file with {build_ini} buildings')
    #buildings_gdf.to_csv('buildings_ini.csv')

    # Check for completely empty columns
    empty_columns = buildings_gdf.columns[buildings_gdf.isna().all()].tolist()
    if empty_columns:
        buildings_gdf = buildings_gdf.drop(columns=empty_columns)
        logging.warning(f'Dropped empty columns: {empty_columns}')

    # Check for NaN values
    nan_dict = {}
    for row_index in range(len(buildings_gdf)):
        row = buildings_gdf.loc[row_index].isna() # checks each column in the specified row for NaN values
        nan_col = row[row].index.tolist()  # Get columns with NaN
        if nan_col:
            id_building = buildings_gdf['id_building'].loc[row_index]
            nan_dict[id_building] = nan_col  # Add id_building and NaN columns to dictionary

    # Drop buildings with NaN values
    # Get the list of id_building values to drop from nan_dict
    ids_to_drop = list(nan_dict.keys())
    # Drop rows where id_building is in ids_to_drop
    buildings_gdf = buildings_gdf[~buildings_gdf['id_building'].isin(ids_to_drop)]
    build_drop = build_ini - buildings_gdf.shape[0]
    logging.warning(f'{build_drop} buildings with NaN values dropped')

    # Extract the CRS
    crs_ini = buildings_gdf.crs

    if crs_ini != crs:
        buildings_gdf = buildings_gdf.to_crs(crs)
        logging.warning(f'Converted CRS from {crs_ini} to {crs}')
    else:
        logging.info(f'CRS: {crs_ini}')

    return buildings_gdf

def apply_grid_mask(buildings_gdf,grid_mask):
        """
        Function to apply a grid mask to a GeoDataFrame of buildings.
        Parameters:
            buildings_gdf: GeoDataFrame with buildings
            grid_mask: GeoDataFrame with the grid
        Returns:
            buildings_gdf: GeoDataFrame with buildings inside the grid mask
        """

        # ensure both GeoDataFrames use the same CRS
        if buildings_gdf.crs != grid_mask.crs:
            crs_ini = buildings_gdf.crs
            buildings_gdf = buildings_gdf.to_crs(grid_mask.crs)
            logging.warning(f'Converted buildings CRS from {crs_ini} to {buildings_gdf.crs}')

        # Compute centroids of the buildings
        centroids = buildings_gdf.copy()
        centroids["geometry"] = buildings_gdf.geometry.centroid

        # Perform spatial join using "within" to keep buildings whose centroids are inside the grid mask
        filtered_centroids = gpd.sjoin(centroids, grid_mask, predicate="within", how="inner")

        # Select original building geometries that correspond to the filtered centroids
        filtered_gdf = buildings_gdf.loc[filtered_centroids.index]

        # Ensure the result is a GeoDataFrame
        filtered_buildings_gdf = gpd.GeoDataFrame(filtered_gdf, geometry='geometry', crs=buildings_gdf.crs)

        return filtered_buildings_gdf

grid = generate_gdf(grid_path,'x','y',2056)
#grid.to_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/grid.gpkg', driver='GPKG')
#grid_mask = generate_grid_mask(grid, 100,2056)
#buidings_cleaned = clean_qbuildings(buildings_path,2056)
#filtered_buildings = apply_grid_mask(buidings_cleaned,grid_mask)
#grid_mask.to_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/grid_mask.gpkg', driver='GPKG')
#filtered_buildings.to_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/buildings_SDEWES.gpkg', driver='GPKG')
#filtered_buildings.to_csv('/Users/catarina/Documents/REHO/scripts/Pathways/QBuildings/buildings_SDEWES.csv')

lausanne_mask = gpd.read_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/Weather/CRYOS_data/Lausanne mask.gpkg')
#buildings_lausanne = apply_grid_mask(buidings_cleaned,lausanne_mask)
#buildings_lausanne.to_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/data/buildings_lausanne.gpkg', driver='GPKG')
#buildings_lausanne.to_csv('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/data/buildings_lausanne.csv')
filtered_buildings = gpd.read_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/data/buildings_SDEWES.gpkg')

grid_mask = gpd.read_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/data/grid_mask.gpkg')
fig_grid, ax = plt.subplots()
#filtered_buildings.plot(ax=ax, color='blue', markersize=1)
filtered_buildings.plot(ax=ax, color='blue', markersize=1)
lausanne_mask.plot(ax=ax, edgecolor='grey', facecolor='none')
grid.plot(ax=ax, color='red', markersize=1)
grid_mask.plot(ax=ax, edgecolor='black', facecolor='none')

ax.set_xlabel('Longitude [°]')
ax.set_ylabel('Latitude [°]')
plt.show()

# Add installed PV

In [ ]:
# Import packages
import os
import logging
from pathlib import Path
import geopandas as gpd
import fiona
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import unary_union
import matplotlib.pyplot as plt

logging.basicConfig(level=logging.INFO,
                    format='[%(levelname)s] %(funcName)s: %(message)s',
                    )

electricity_plants_path = '/Users/catarina/Library/CloudStorage/OneDrive-epfl.ch/1.Projects & Colaborations/UrbanTwin/18.Data/Electricity generation plants/052025_ch.bfe.elektrizitaetsproduktionsanlagen.gpkg'
electricity_csv_path = '/Users/catarina/Library/CloudStorage/OneDrive-epfl.ch/1.Projects & Colaborations/UrbanTwin/18.Data/Electricity generation plants/electricity_production_plants.csv'

plant_df = pd.read_csv(electricity_csv_path)

# list all layers in the file
layers = fiona.listlayers(electricity_plants_path)

# Read each layer separately into a GeoDataFrame
gdfs = {layer: gpd.read_file(electricity_plants_path, layer=layer) for layer in layers}

lausanne_mask = gpd.read_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/Weather/CRYOS_data/Lausanne mask.gpkg')

# ensure both GeoDataFrames use the same CRS
if gdfs['ElectricityProductionPlant'].crs != lausanne_mask.crs:
    crs_ini = gdfs['ElectricityProductionPlant'].crs
    gdfs['ElectricityProductionPlant'] = gdfs['ElectricityProductionPlant'].to_crs(lausanne_mask.crs)
    logging.warning(f'Converted plant CRS from {crs_ini} to {gdfs["ElectricityProductionPlant"].crs}')

# Add a 2-meter buffer to the lausanne_mask (expanding it by 100 meters), to include the plants that are very close to the border
# Create a buffered version of the lausanne_mask
lausanne_mask_buffered = lausanne_mask.buffer(100)
# Combine buffered geometries into a single shape (if needed)
lausanne_mask_buffered = gpd.GeoDataFrame(geometry=[lausanne_mask_buffered.unary_union], crs=lausanne_mask.crs)

# Perform spatial join using "within" to keep plants inside the grid mask. Ignore index of lausanne_mask_buffered
filtered_plants = gpd.sjoin(gdfs['ElectricityProductionPlant'], lausanne_mask_buffered, predicate="within", how="inner")

# Drop the 'index_right' column added by the join
filtered_plants = filtered_plants.drop(columns='index_right')

fig_grid, ax = plt.subplots()
#filtered_buildings.plot(ax=ax, color='blue', markersize=1)
lausanne_mask.plot(ax=ax, edgecolor='grey', facecolor='none')
lausanne_mask_buffered.plot(ax=ax, edgecolor='black', facecolor='none')
#grid.plot(ax=ax, color='red', markersize=1)
filtered_plants.plot(ax=ax, color='green', markersize=1)

ax.set_xlabel('Longitude [°]')
ax.set_ylabel('Latitude [°]')
plt.show()

# get dictionaries from the other layers
# convert dataframe to dictionary
MainCategory_df = gdfs['MainCategoryCatalogue']
# keep columns 'Catalogue_id and 'en'
MainCategory_df = MainCategory_df[['Catalogue_id', 'en']]

MainCategory_dict = {}
for index, row in MainCategory_df.iterrows():
    # Create a dictionary entry with 'Catalogue_id' as key and 'en' as value
    MainCategory_dict[row['Catalogue_id']] = row['en']

SubCategory_df = gdfs['SubCategoryCatalogue']
# keep columns 'Catalogue_id and 'en'
SubCategory_df = SubCategory_df[['Catalogue_id', 'en']]
SubCategory_dict = {}
for index, row in SubCategory_df.iterrows():
    # Create a dictionary entry with 'Catalogue_id' as key and 'en' as value
    SubCategory_dict[row['Catalogue_id']] = row['en']

PlantCategory_df = gdfs['PlantCategoryCatalogue']
# keep columns 'Catalogue_id and 'en'
PlantCategory_df = PlantCategory_df[['Catalogue_id', 'en']]
# Create a dictionary entry with 'Catalogue_id' as key and 'en' as value
PlantCategory_dict = {}
for index, row in PlantCategory_df.iterrows():
    # Create a dictionary entry with 'Catalogue_id' as key and 'en' as value
    PlantCategory_dict[row['Catalogue_id']] = row['en']

filtered_plants['MainCategory'] = filtered_plants['MainCategory'].map(MainCategory_dict)
filtered_plants['SubCategory'] = filtered_plants['SubCategory'].map(SubCategory_dict)
filtered_plants['PlantCategory'] = filtered_plants['PlantCategory'].map(PlantCategory_dict)

# Get the InitialPower and TotalPower from the plant_df
filtered_plants['InitialPower'] = filtered_plants['xtf_id'].map(plant_df.set_index('xtf_id')['InitialPower'])
filtered_plants['TotalPower'] = filtered_plants['xtf_id'].map(plant_df.set_index('xtf_id')['TotalPower'])

# filter the rows to keep only the ones with the SubCategory 'Photovoltaic'
pv_plants = filtered_plants[filtered_plants['SubCategory'] == 'Photovoltaic']

# Save the filtered plants to a geopackage file
filtered_plants.to_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/data/electricity_plants_Lausanne.gpkg', driver='GPKG')
# Save the PV plants to a geopackage file
pv_plants.to_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/data/Lausanne_PV.gpkg', driver='GPKG')
pv_plants.to_csv('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/data/Lausanne_PV.csv')


## Add installed PV to QBuildings

In [ ]:
import geopandas as gpd
import pandas as pd

# read data
buildings_df = gpd.read_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/data/buildings_SDEWES.gpkg')
pv_plants_df = gpd.read_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/data/Lausanne_PV.gpkg')

# Ensure both GeoDataFrames use the same coordinate reference system
pv_plants_df = pv_plants_df.to_crs(buildings_df.crs)

# Spatial join: find which PV points fall within which buildings
pv_with_buildings = gpd.sjoin(pv_plants_df, buildings_df, how='inner', predicate='within')

# Sum PV power per building. Adjust 'pv_power_kW' to your actual power column name
pv_power_by_building = pv_with_buildings.groupby('id_building')['TotalPower'].sum().reset_index()

# Set up a dictionary for quick lookup of PV power per building
pv_power_dict = pv_power_by_building.set_index('id_building')['TotalPower'].to_dict()

# Set is_pv to True where the building has PV
buildings_df['is_pv'] = buildings_df['id_building'].isin(pv_power_dict)

# Set pv_power using the dictionary, use .get() to handle buildings with no PVs (default to 0)
buildings_df['pv_power_kW'] = buildings_df['id_building'].map(pv_power_dict).fillna(0)

# Save the updated buildings GeoDataFrame to a new file
buildings_df.to_file('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/data/buildings_SDEWES_with_pv.gpkg', driver='GPKG')
# Save the updated buildings GeoDataFrame to a CSV file
buildings_df.to_csv('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/data/buildings_SDEWES_with_pv.csv')


